# Mutual Information Check With Features and Target

## 1- Initial Preprocessing

In [1]:
import numpy as np
import pandas as pd

import sys
import os
sys.path.append(os.path.abspath('../'))

from Models.LinearRegression import LinearRegression
from Utils.Preprocessor import Preprocessor
from Utils.Utils import root_mean_squared_error, train_test_split, initial_preprocessing

In [2]:
# Read the data
train = pd.read_csv('../Data/train.csv', index_col='Id')

In [3]:
# Remove unnecessary features based on exploratory data analysis part 1.
train = initial_preprocessing(train)

In [4]:
X = train.drop(columns=["num_wins_agent1", "num_draws_agent1", "num_losses_agent1", "utility_agent1"], axis=1)
y = train["utility_agent1"]
y1 = train["num_wins_agent1"]
y2 = train["num_draws_agent1"]
y3 = train["num_losses_agent1"]

In [5]:
# Preprocess the data
preprocessor = Preprocessor(normalize=True, one_hot_encode=True)

X = preprocessor.fit_transform(X)

# Convert back to pandas dataframe
X = pd.DataFrame(X, columns=preprocessor.get_column_names())

## 2- Mutual Information Check with Target

### 2.1- utility_agent1

In [6]:
def calculate_mutual_information(X, y):
    # taken the formula from internet dont know if works.
    from collections import Counter
    
    def entropy(values):
        """Calculate the entropy of a dataset."""
        total = len(values)
        counts = Counter(values)
        probabilities = np.array([count / total for count in counts.values()])
        return -np.sum(probabilities * np.log2(probabilities + 1e-9))

    def conditional_entropy(feature, target):
        """Calculate the conditional entropy of target given feature."""
        total = len(feature)
        unique_values = np.unique(feature)
        cond_entropy = 0
        for value in unique_values:
            indices = np.where(feature == value)[0]
            subset = target[indices]
            cond_entropy += len(subset) / total * entropy(subset)
        return cond_entropy

    # Drop categorical columns
    X = X.drop(columns=X.select_dtypes(include=['category']).columns, axis=1)
    
    # Store column names for later data frame creation
    colnames = X.columns
    
    # Convert to numpy arrays
    X = X.values
    y = y.values if isinstance(y, pd.Series) else y
    
    # Calculate mutual information for each feature
    mutual_info = []
    for i in range(X.shape[1]):
        feature = X[:, i]
        feature_entropy = entropy(feature)
        target_entropy = entropy(y)
        cond_entropy = conditional_entropy(feature, y)
        mi = target_entropy - cond_entropy
        mutual_info.append(mi)
    
    # Create a DataFrame with results
    mutual_info_df = pd.DataFrame({
        "Feature": colnames,
        "Mutual Information": mutual_info
    }).sort_values(by="Mutual Information", ascending=False)
    
    return mutual_info_df

In [7]:
mutual_info_utility_agent1 = calculate_mutual_information(X, y)

In [8]:
mutual_info_utility_agent1.head(10)

,Feature,Mutual Information
549,MovesPerSecond,1.283681
548,PlayoutsPerSecond,1.280695
376,DurationActions,1.273614
377,DurationMoves,1.263127
382,GameTreeComplexity,1.262383
378,DurationTurns,1.252940
380,DurationTurnsNotTimeouts,1.246248
379,DurationTurnsStdDev,1.182272
406,BranchingFactorVariance,1.117977
416,DecisionFactorVariance,1.091581


### 2.1- num_wins_agent1

In [9]:
mutual_info_num_wins_agent1 = calculate_mutual_information(X, y1)

In [10]:
mutual_info_num_wins_agent1.head(10)

,Feature,Mutual Information
549,MovesPerSecond,0.986466
548,PlayoutsPerSecond,0.983382
376,DurationActions,0.978639
377,DurationMoves,0.970353
382,GameTreeComplexity,0.968839
378,DurationTurns,0.961348
380,DurationTurnsNotTimeouts,0.956689
379,DurationTurnsStdDev,0.908221
406,BranchingFactorVariance,0.850977
416,DecisionFactorVariance,0.836046


### 2.2- num_draws_agent1

In [11]:
mutual_info_num_draws_agent1 = calculate_mutual_information(X, y2)

In [12]:
mutual_info_num_draws_agent1.head(10)

,Feature,Mutual Information
549,MovesPerSecond,1.041842
548,PlayoutsPerSecond,1.037821
376,DurationActions,1.036927
382,GameTreeComplexity,1.028330
377,DurationMoves,1.027954
378,DurationTurns,1.019944
380,DurationTurnsNotTimeouts,1.010021
379,DurationTurnsStdDev,0.963085
406,BranchingFactorVariance,0.914733
416,DecisionFactorVariance,0.906778


### 2.3- num_losses_agent1

In [13]:
mutual_info_num_lossess_agent1 = calculate_mutual_information(X, y3)

In [14]:
mutual_info_num_lossess_agent1.head(10)

,Feature,Mutual Information
549,MovesPerSecond,0.982406
548,PlayoutsPerSecond,0.979751
376,DurationActions,0.973548
377,DurationMoves,0.967428
382,GameTreeComplexity,0.964320
378,DurationTurns,0.962127
380,DurationTurnsNotTimeouts,0.957681
379,DurationTurnsStdDev,0.912563
406,BranchingFactorVariance,0.843829
416,DecisionFactorVariance,0.826475
